<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/total_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seeing the imperceptible: total flow

this is a notebook which is used to have a set of images move through the processing steps which can individually be found in the topic related folders.

A **batch** of images will be processed and all intermediate steps will be saved. This is not an optimized flow, yet a flow allowing the user to intervene where necessary and have updated images continue throughout the flow.

This flow is created to test with some experts and capture their feedback, it is not intended for day-to-day usage. One testing day will be organized in which this notebook will be used with some unique images.

## steps

1. Sensor dust removal
2. Image Editing (LaMa)
3. Background Removal
4. Background Recoloring: not yet
5. Clothes recoloring: not yet
6. Skin retouching
<!-- 7. Face Detection -->
7. optional: Color Corrections: not yet
8. Color Grading: 
9. Image upscaling

TODO: is it necessary to create a config file and have separate notebooks for each step? (with 1 common config generator)

In [1]:
# first we'll link a database connection:
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py --silent
from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

what's the username for nextcloud? colab
what's the password for user colab? ··········
0
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  


## SETUP

we'll link this instance of the machine learning flow to your name:

In [2]:
tname = 'total' #@param {type:"string"}
if(tname=='total'):
    print("Are you sure you don't want to change the name?")

Are you sure you don't want to change the name?


## Step 1: Sensor dust removal

we'll link the main input folder and write the output images in the output folder of step 1.

In [3]:
#@title imports of libraries & setting up
input_step1, output_step1 = create_io(database=nextcloud,topic=tname,library='step1_sensor_dust', input_redirect='/content/database/total/input')

#import libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os, sys
!curl https://raw.githubusercontent.com/Tschucker/Python-Automatic-Sensor-Dust-Removal/main/shapedetector.py -o /content/shapedetector.py
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
from shapedetector import ShapeDetector
import imutils
from google.colab.patches import cv2_imshow

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1255  100  1255    0     0   7212      0 --:--:-- --:--:-- --:--:--  7212


In [4]:
#@title Set inpainting options and run the model
radius = 11 #@param {type:"slider",min:1, max:50}
flags = cv2.INPAINT_TELEA #@param ["cv2.INPAINT_TELEA","cv2.INPAINT_NS"]

def inpaint_img(img_path, img_name, output_path, radius=10, flags=cv2.INPAINT_TELEA):
  #color version
  cimg = cv2.imread(img_path)
  #grey scale image
  img = cv2.imread(img_path,0)

  #Apply Global Threshold
  m = np.mean(img, dtype=int)
  global_thresh = cv2.threshold(img,int(m/1.2),255,cv2.THRESH_BINARY_INV)[1]

  #Perform Adaptive Threshold
  adaptive_thresh_img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,19,10)

  #Image Magnification Filter Kernel
  KERNEL = np.ones((10,10), dtype=int)*10

  #Filter the thresholded images*
  img_filt = cv2.filter2D(adaptive_thresh_img,-1,KERNEL)
  #global_thresh = cv2.filter2D(global_thresh,-1,KERNEL)

  #Apply multiple times
  for i in range(2):
      KERNEL_i = np.ones((int(10),int(10)), dtype=int)*10
      img_filt = cv2.filter2D(img_filt,-1,KERNEL_i)

  #Combine Thresholds
  comb = img_filt + global_thresh

  #Find and Classify Contours of Image
  cnts = cv2.findContours(comb.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  sd = ShapeDetector()
  cimg_copy = cimg.copy()
  for c in cnts:
      # compute the center of the contour, then detect the name of the
      # shape using only the contour
      M = cv2.moments(c)
      if M["m00"] != 0:
          cX = int((M["m10"] / M["m00"]) * 1)
          cY = int((M["m01"] / M["m00"]) * 1)
          shape = sd.detect(c)
          # multiply the contour (x, y)-coordinates by the resize ratio,
          # then draw the contours and the name of the shape on the image
          if len(c) < 50:
              c = c.astype("float")
              c *= 1
              c = c.astype("int")
              cv2.drawContours(cimg_copy, [c], -1, (0, 255, 0), 2)
              cv2.putText(cimg_copy, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
  
  #Create Dust Mask
  img_mask = np.zeros((img.shape[0], img.shape[1]), dtype='uint8')
  for c in cnts:
      # compute the center of the contour, then detect the name of the
      # shape using only the contour
      M = cv2.moments(c)
      if M["m00"] != 0:
          cX = int((M["m10"] / M["m00"]) * 1)
          cY = int((M["m01"] / M["m00"]) * 1)
          shape = sd.detect(c)
          # multiply the contour (x, y)-coordinates by the resize ratio,
          # then draw the contours and the name of the shape on the image
          if len(c) < 50:
              c = c.astype("float")
              c *= 1
              c = c.astype("int")
              cv2.fillPoly(img_mask, pts=[c], color=(255,255,255))

    
  #Inpaint the image
  cimg_inpaint = cv2.inpaint(cimg, img_mask, radius, flags=flags)

  #Show and Save Final Image
  save_img_pth = os.path.join(output_path,img_name)
  cv2.imwrite(save_img_pth, cimg_inpaint)

  # plt_out = cv2.cvtColor(cimg_inpaint, cv2.COLOR_BGR2RGB)
  # return plt_out

for img_name in os.listdir(input_step1):
  print("processing ",img_name)
  img_path = os.path.join(input_step1,img_name)
  inpaint_img(img_path, img_name, output_step1, radius=radius, flags=flags)

processing  1.jpg
processing  2.jpg


### verification

Now it's time to go to the database and verify the results. If needed we adapt the images locally. 

TODO: how easily can we **sync** the images to allow the experts to intervene? Is it possible with nextcloud?

## step 2: Minor retouching: image editing LaMa 
Once verified we want to continue with the next step, being image editing (LaMa model)

therefore we link the output folder of previous step to the inputfolder of this step.

In [5]:
#@title imports of libraries & setting up

input_step2, output_step2 = create_io(database=nextcloud,topic=tname,library='step2_lama', input_redirect=output_step1)
root_path2 = '/content/lama'

# clone the repository
if not os.path.exists(root_path2):
  !git clone https://github.com/saic-mdal/lama {root_path2}
# Set up the environment
print('\n> Install dependencies')
!pip install -qr lama/requirements.txt 
# do we need the wget?
!pip install -q wget 

# download the model
print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o {root_path2}/big-lama.zip
# todo check where the model is unzipped
!unzip {root_path2}/big-lama.zip -d {root_path2}
# fixing openCV
print('>fixing opencv')
!pip uninstall -q opencv-python-headless -y 
!pip install -q opencv-python-headless==4.1.2.30 

Cloning into '/content/lama'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 309 (delta 47), reused 40 (delta 39), pack-reused 240
Receiving objects: 100% (309/309), 6.51 MiB | 20.13 MiB/s, done.
Resolving deltas: 100% (89/89), done.

> Install dependencies
     |████████████████████████████████| 12.5 MB 5.3 MB/s 
     |████████████████████████████████| 22.3 MB 1.7 MB/s 
     |████████████████████████████████| 72 kB 533 kB/s 
     |████████████████████████████████| 144 kB 50.7 MB/s 
     |████████████████████████████████| 841 kB 35.8 MB/s 
     |████████████████████████████████| 271 kB 51.0 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 948 kB 42.9 MB/s 
     |████████████████████████████████| 47.8 MB 48 kB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 112 kB 39.3 MB/s 
     |█

In [6]:
#@title imports & helper functions

import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

canvas_html = """
<style>
.button {
  background-color: #4CAF50;
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;
}
</style>
<canvas1 width=%d height=%d>
</canvas1>
<canvas width=%d height=%d>
</canvas>

<button class="button">Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')

var canvas1 = document.querySelector('canvas1')
var ctx1 = canvas.getContext('2d')


ctx.strokeStyle = 'red';

var img = new Image();
img.src = "data:image/%s;charset=utf-8;base64,%s";
console.log(img)
img.onload = function() {
  ctx1.drawImage(img, 0, 0);
};
img.crossOrigin = 'Anonymous';

ctx.clearRect(0, 0, canvas.width, canvas.height);

ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}

canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}

var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(imgm, filename='drawing.png', w=400, h=200, line_width=1):
  display(HTML(canvas_html % (w, h, w,h, filename.split('.')[-1], imgm, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)


In [11]:
#@title some error happens in LaMa, so we're skipping this step for now
import shutil
for file in os.listdir(input_step2):
  shutil.copy2(os.path.join(input_step2, file), output_step2)

# for i in os.listdir(input_step2):
#   fname = os.path.join(input_step2,i)

#   image64 = base64.b64encode(open(fname, 'rb').read())
#   image64 = image64.decode('utf-8')

#   print(f'Will use {fname} for inpainting')
#   img = np.array(plt.imread(f'{fname}')[:,:,:3])

#   draw(image64, filename=f"./{fname.split('.')[1]}_mask.png", w=img.shape[1], h=img.shape[0], line_width=0.04*img.shape[1])
#   #@title Show a masked image and save a mask

#   # plt.rcParams["figure.figsize"] = (15,5)
#   # plt.rcParams['figure.dpi'] = 200
#   # plt.subplot(131)
#   # with_mask = np.array(plt.imread(f"./{fname.split('.')[1]}_mask.png")[:,:,:3])
#   # mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)
#   # plt.imshow(mask, cmap='gray')
#   # plt.axis('off')
#   # plt.title('mask')
#   # plt.imsave(f"./{fname.split('.')[1]}_mask.png",mask, cmap='gray')

#   # plt.subplot(132)
#   # img = np.array(plt.imread(f'{fname}')[:,:,:3])
#   # plt.imshow(img)
#   # plt.axis('off')
#   # plt.title('img')

#   # plt.subplot(133)
#   # img = np.array((1-mask.reshape(mask.shape[0], mask.shape[1], -1))*plt.imread(fname)[:,:,:3])
#   # _=plt.imshow(img)
#   # _=plt.axis('off')
#   # _=plt.title('img * mask')
#   # plt.show()

#   # os.makedirs('/content/output')

#   print('Run inpainting')
#   !echo $(pwd)
#   if '.jpeg' in fname:
#     !PYTHONPATH={root_path2} TORCH_HOME={root_path2} python3 {root_path2}/bin/predict.py model.path={root_path2}/big-lama indir={input_step2} outdir={output_step2} dataset.img_suffix=.jpeg > /dev/null
#   elif '.jpg' in fname:
#     !PYTHONPATH={root_path2} TORCH_HOME=$(pwd) python3 {root_path2}/bin/predict.py model.path={root_path2}/big-lama indir={input_step2}  outdir={output_step2}  dataset.img_suffix=.jpg   > /dev/null
#   elif '.png' in fname:
#     !PYTHONPATH={root_path2} TORCH_HOME={root_path2} python3 {root_path2}/bin/predict.py model.path={root_path2}/big-lama indir={input_step2}  outdir={output_step2}  dataset.img_suffix=.png > /dev/null
#   else:
#     print(f'Error: unknown suffix .{fname.split(".")[-1]} use [.png, .jpeg, .jpg]')

#   plt.rcParams['figure.dpi'] = 200

#   print(os.listdir(output_step2))

# # plt.imshow(plt.imread(f"/content/output/{fname.split('.')[1].split('/')[2]}_mask.png"))
# # _=plt.axis('off')
# # _=plt.title('inpainting result')
# # plt.show()

# # fname = None



## Step 3: background removal

Again, verify the outcome of step 2. 

Now we'll subtract the background using the PaddleSeg model



In [12]:
#@title
input_step3, output_step3 = create_io(database=nextcloud,topic=tname,library='step3_bg_removal', input_redirect=output_step2)

In [16]:
#@title imports for paddleseg
!pip install -q PaddlePaddle
root_path = '/content/PaddleSeg'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/PaddlePaddle/PaddleSeg {root_path}

%cd {root_path}
!pip -qq install -r requirements.txt'
!pip install -e .

# installing Matting
%cd Matting
!pip -qq install -r requirements.txt

/content/PaddleSeg
Obtaining file:///content/PaddleSeg
  Attempting uninstall: paddleseg
    Found existing installation: paddleseg 2.5.0
    Can't uninstall 'paddleseg'. No files were found to uninstall.
  Running setup.py develop for paddleseg
/content/PaddleSeg/Matting


In [17]:
#@title downloading the models
# download model checkpoint 
model_path = root_path + '/Matting/data/model'
model_params = 'https://paddleseg.bj.bcebos.com/matting/models/human_matting-resnet34_vd.pdparams'
model_inf = 'https://paddleseg.bj.bcebos.com/matting/models/deploy/pp-humanmatting-resnet34_vd.zip'
# make folders
os.makedirs(model_path, exist_ok=True)
if not os.path.exists(os.path.join(model_path,'human_matting.pdparams')):
  print('\n> Download the model params')
  !curl {model_params} -o {os.path.join(model_path,'human_matting.pdparams')}
else:
  print ('\n> File already downloaded')
if not os.path.exists(os.path.join(model_path,'human_matting-resnet.zip')):
  print('\n> Download the model')
  !curl {model_inf} -o {os.path.join(model_path,'human_matting-resnet.zip')}
  !unzip -q {os.path.join(model_path,'human_matting-resnet.zip')} -d {os.path.join(model_path)}
else:
  print ('\n> File already downloaded')


> Download the model params
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  243M  100  243M    0     0  4691k      0  0:00:53  0:00:53 --:--:-- 12.8M

> Download the model
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  226M  100  226M    0     0  4911k      0  0:00:47  0:00:47 --:--:-- 10.0M


In [23]:
mode = 'bgremoval' #@param ['predict','bgremoval']
if(mode=='predict'):
# predict
  !export CUDA_VISIBLE_DEVICES=0
  !python predict.py \
      --config configs/human_matting/human_matting-resnet34_vd.yml \
      --model_path data/model/human_matting.pdparams \
      --image_path {input_step3} \
      --save_dir {output_step3} \
      --fg_estimate True
else:
  # bg replacement
  background = 'r' #@param ['r','g','b','w'] {allow-input: true}
  !export CUDA_VISIBLE_DEVICES=0
  !python bg_replace.py \
      --config configs/human_matting/human_matting-resnet34_vd.yml \
      --model_path data/model/human_matting.pdparams \
      --image_path {input_step3} \
      --background r \
      --save_dir {output_step3} \
      --fg_estimate True

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

## Step 4: Clothes recoloring

In [ ]:
input_step4, output_step4 = create_io(database=nextcloud,topic=tname,library='step5_clothes_coloring', input_redirect=output_step3)

In [ ]:
import shutil
# as we're not implementing clothes recoloring yet, we copy the folders to surpass this step
for file in os.listdir(input_step4):
  shutil.copy2(os.path.join(input_step4, file), output_step4)

## Step 6: Skin retouching

we'll implement the retouchML library

In [ ]:
input_step6, output_step6 = create_io(database=nextcloud,topic=tname,library='step6_skin_retouch', input_redirect=output_step5)

## Step 7: Color Corrections



In [ ]:
input_step7, output_step7 = create_io(database=nextcloud,topic=tname,library='step6_color_corrections', input_redirect=output_step6)

## Step 8: Color Grading

we're implementing the deep preset library.

In [ ]:
input_step8, output_step8 = create_io(database=nextcloud,topic=tname,library='step7_color_grading', input_redirect=output_step7)

## Step 9: image upscaling

This is to be implemented **if** necessary?

In [ ]:
input_step9, output_step9 = create_io(database=nextcloud,topic=tname,library='step8_image_upscaling', input_redirect=output_step8)